Analisis Exploratorio de Datos: EDA

In [13]:
import numpy as np
import yfinance as yf
import pandas as pd
#import matplotlib.pyplot as plt


In [10]:
sp500_data = yf.download('^GSPC', start='2000-01-01')


[*********************100%***********************]  1 of 1 completed


In [15]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].tolist()

In [17]:
historical_prices = yf.download(tickers, start='2000-01-01', end='2023-02-23')

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
- BRK.B: No timezone found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


In [18]:
historical_prices

Adj Close                                                 \
                     A        AAL         AAP        AAPL        ABBV   
Date                                                                    
2000-01-03   44.001263        NaN         NaN    0.850643         NaN   
2000-01-04   40.640060        NaN         NaN    0.778926         NaN   
2000-01-05   38.119148        NaN         NaN    0.790324         NaN   
2000-01-06   36.667713        NaN         NaN    0.721931         NaN   
2000-01-07   39.723362        NaN         NaN    0.756127         NaN   
...                ...        ...         ...         ...         ...   
2023-02-15  151.750000  16.799999  151.289993  155.330002  151.059998   
2023-02-16  148.279999  16.389999  150.190002  153.710007  149.529999   
2023-02-17  148.259995  16.360001  149.990005  152.550003  151.309998   
2023-02-21  143.419998  15.690000  144.190002  148.479996  152.000000   
2023-02-22  142.080002  15.560000  143.619995  148.910004  150.669998   

                                                                       ...  \
                   ABC         ABT       ACGL         ACN        ADBE  ...   
Date                                                                   ...   
2000-01-03    2.988898    9.221025   1.277778         NaN   16.274670  ...   
2000-01-04    2.784837    8.957567   1.270833         NaN   14.909400  ...   
2000-01-05    3.000900    8.941105   1.388889         NaN   15.204173  ...   
2000-01-06    3.228970    9.253959   1.375000         NaN   15.328288  ...   
2000-01-07    3.649096    9.352759   1.451389         NaN   16.072987  ...   
...                ...         ...        ...         ...         ...  ...   
2023-02-15  158.809998  107.279999  67.980003  287.649994  375.940002  ...   
2023-02-16  160.470001  106.080002  67.099998  281.220001  365.160004  ...   
2023-02-17  161.490005  106.739998  67.809998  277.049988  356.850006  ...   
2023-02-21  160.839996  103.650002  67.550003  269.149994  346.700012  ...   
2023-02-22  160.130005  102.690002  67.940002  269.630005  348.720001  ...   

               Volume                                                  \
                 WYNN      XEL       XOM     XRAY        XYL      YUM   
Date                                                                    
2000-01-03        NaN  2738600  13458200   582300        NaN  3033493   
2000-01-04        NaN   425200  14510800   317700        NaN  3315031   
2000-01-05        NaN   500200  17485000  1188000        NaN  4642602   
2000-01-06        NaN   344100  19461600   534300        NaN  3947658   
2000-01-07        NaN   469500  16603800  1401000        NaN  6063647   
...               ...      ...       ...      ...        ...      ...   
2023-02-15  1766700.0  2810200  13673400  2488000   956800.0  1456800   
2023-02-16  2026300.0  2870600  11360600  1704300  1091500.0  1922700   
2023-02-17  2708300.0  2308000  20752900  1334300  1157200.0  1253000   
2023-02-21  1894500.0  2664000  14469400  1574100  1568600.0  1739800   
2023-02-22  3375000.0  2079500  13968100  1195900  1498800.0  1977100   

                                                    
                  ZBH     ZBRA     ZION        ZTS  
Date                                                
2000-01-03        NaN  1055700  1199600        NaN  
2000-01-04        NaN   522450   816100        NaN  
2000-01-05        NaN   612225  1124700        NaN  
2000-01-06        NaN   263925  1112100        NaN  
2000-01-07        NaN   333900   782000        NaN  
...               ...      ...      ...        ...  
2023-02-15  1022200.0   305600   951500  2510800.0  
2023-02-16  1410900.0   555800   935000  1951100.0  
2023-02-17  1026800.0   496800  1120200  1954500.0  
2023-02-21  1266800.0   375000  1364900  1791300.0  
2023-02-22  1260100.0   309000  1301100  2103800.0  

[5822 rows x 3018 columns]